In [1]:
# if you are running in the notebooks folder example cloned from github this line must be included
import sys
import os
sys.path.insert(0, os.path.abspath('../'))

import numpy as np

#Propagator engine
import poliastro

#The library
import CtllDes 
from CtllDes.core import ctll, satellite

#Throughout this examples it will be necessary to use the correct
#physical dimensions, astropy.units will help us with that
import astropy.units as u

In [2]:
#Orbit object, more info on poliastro API reference.
from poliastro.twobody import Orbit

#The bodies module specifies attractors
from poliastro.bodies import Earth


#Planes of reference and Epochs, are almost every time 
#the default ones, but just in case if you want to use 
#any other, they can be found in the following modules

#import planes of reference, EARTH_EQUATOR 
from poliastro.frames import Planes

#import J2000 time reference from constants 
from poliastro.constants import J2000

### Specify classical orbit parameters ###

a = 8000 * u.km # semi-major axis [distance]
ecc = 0 * u.one # eccentricity [dimensionleess]
inc = 85 * u.deg # inclination [angle] 
raan = 0 * u.rad # right ascencion of the ascending node [angle]
argp = 0 * u.rad # perigee argument [angle]
nu = 0 * u.rad # true anomaly [angle]

plane = Planes.EARTH_EQUATOR # not necessary to specify
epoch = J2000 # not necessary to specify

#classmethod of Orbit
orb = Orbit.from_classical(Earth,
                            a,
                            ecc,
                            inc,
                            raan,
                            argp,
                            nu)

In [3]:
from CtllDes.requests.coverage import Coverages

In [4]:
from CtllDes.targets.targets import Target, Targets

In [5]:
sat = satellite.Sat.from_orbit(orb)

In [92]:
from CtllDes.core.instrument import Instrument

from CtllDes.requests.coverage import symmetric_with_roll

class RollCamera(Instrument):
    def __init__(self,FOV,roll_angle):
        """Constructor for RollCamera.
        
        Parameters
        ----------
        FOV : ~astropy.units.quantity.Quantity
            field of view, angle quantity
        roll_angle : ~astropy.units.quantity.Quantity
            maximum rolling angle
        """
        
        super().__init__()
        self.FOV = FOV.to(u.rad)
        self.roll = roll_angle.to(u.rad)
        
    def coverage(self, lons, lats, r, v, target, R):
        return symmetric_with_roll(self.FOV,
                                      lons,
                                      lats,
                                      r,
                                      v,
                                      target,
                                      R,
                                      roll_angle = self.roll)


rollcam = RollCamera(15*u.deg,10*u.deg)
sat.update_instruments(rollcam,f=True)

In [61]:
cov = Coverages.from_sat(sat, Targets([Target(10,5*i) for i in range(0,19)]), 20, dt=7, J2=True )

target 10.00° 0.00°. 1 of 19
target 10.00° 5.00°. 2 of 19
target 10.00° 10.00°. 3 of 19
target 10.00° 15.00°. 4 of 19
target 10.00° 20.00°. 5 of 19
target 10.00° 25.00°. 6 of 19
target 10.00° 30.00°. 7 of 19
target 10.00° 35.00°. 8 of 19
target 10.00° 40.00°. 9 of 19
target 10.00° 45.00°. 10 of 19
target 10.00° 50.00°. 11 of 19
target 10.00° 55.00°. 12 of 19
target 10.00° 60.00°. 13 of 19
target 10.00° 65.00°. 14 of 19
target 10.00° 70.00°. 15 of 19
target 10.00° 75.00°. 16 of 19
target 10.00° 80.00°. 17 of 19
target 10.00° 85.00°. 18 of 19
target 10.00° 90.00°. 19 of 19


In [97]:
import matplotlib.pyplot as plt
%matplotlib qt5

In [ ]:
plt.figure(figsize=(10,30))

for covv in cov.covs:
    x = np.array( [covv.dt*i/3660 for i in range(len(covv.cov))] ) 
    plt.step(x, np.array(covv.cov)*3.5 + covv.target.lat,c='orange')

plt.xlabel("time [h]")
plt.ylabel("ground station latitude [°]")
plt.grid(axis='y')
plt.title("Tombstone plot, ground station at fixed longitude = 10 °")

In [75]:
covdf = cov.to_df()

In [80]:
accum = covdf['accumulated'].to_numpy(dtype=np.float64)
accum /= max(accum)

response_time = covdf['response time'].to_numpy(dtype=np.float64)
response_time /= max(response_time)

avg_time_gap = covdf['mean gap dark'].to_numpy(dtype=np.float64)
avg_time_gap /= max(avg_time_gap)

#max_gap = covdf['max gap']


plt.figure(figsize=(10,10))

plt.plot([5*i for i in range(19)], accum, label="Normalized accumulated time")
plt.plot([5*i for i in range(19)], response_time, label="Normalized response time")
plt.plot([5*i for i in range(19)], avg_time_gap,label="Normalized average time gap")
plt.xlabel("latitude [°]")
plt.legend(loc=1)
plt.grid()





In [117]:
roll_covs = []
angles = np.linspace(0,30,30)


In [118]:
for angle in angles:
    rollcam = RollCamera(15*u.deg,angle*u.deg)
    sat.update_instruments(rollcam,f=True)
    roll_covs.append(Coverages.from_sat(sat, Target(10,65), 20, dt=7, J2=True))


target 10.00° 65.00°. 1 of 1
target 10.00° 65.00°. 1 of 1
target 10.00° 65.00°. 1 of 1
target 10.00° 65.00°. 1 of 1
target 10.00° 65.00°. 1 of 1
target 10.00° 65.00°. 1 of 1
target 10.00° 65.00°. 1 of 1
target 10.00° 65.00°. 1 of 1
target 10.00° 65.00°. 1 of 1
target 10.00° 65.00°. 1 of 1
target 10.00° 65.00°. 1 of 1
target 10.00° 65.00°. 1 of 1
target 10.00° 65.00°. 1 of 1
target 10.00° 65.00°. 1 of 1
target 10.00° 65.00°. 1 of 1
target 10.00° 65.00°. 1 of 1
target 10.00° 65.00°. 1 of 1
target 10.00° 65.00°. 1 of 1
target 10.00° 65.00°. 1 of 1
target 10.00° 65.00°. 1 of 1
target 10.00° 65.00°. 1 of 1
target 10.00° 65.00°. 1 of 1
target 10.00° 65.00°. 1 of 1
target 10.00° 65.00°. 1 of 1
target 10.00° 65.00°. 1 of 1
target 10.00° 65.00°. 1 of 1
target 10.00° 65.00°. 1 of 1
target 10.00° 65.00°. 1 of 1
target 10.00° 65.00°. 1 of 1
target 10.00° 65.00°. 1 of 1


In [120]:
roll_covs_df = [r.to_df() for r in roll_covs]
accums = np.array([r['accumulated'].values for r in roll_covs_df])
accums /= max(accums)

response_time = np.array([r['response time'].values for r in roll_covs_df])
response_time /= max(response_time)

revisit = np.array([r['mean gap dark'].values for r in roll_covs_df])
revisit /= max(revisit)

(8, 1)

In [131]:
plt.figure(figsize=(10,10))

plt.plot(angles,accums.reshape(30,), label="Normalized accumulated time"
)
plt.plot(angles,response_time.reshape(30,), label="Normalized response time"
)
plt.plot(angles,revisit.reshape(30,),label="Normalized average time gap" )
plt.legend(loc=1)
plt.xlabel("Maximum roll angle [°]")
plt.grid()

array([0.056365030674846626, 0.09675357873210634, 0.15414110429447853,
       0.22929447852760737, 0.33320552147239263, 0.4730316973415133,
       0.6802147239263804, 1.0], dtype=object)